In [ ]:
from tkinter import Tk, Label, Button, filedialog, messagebox
from PIL import Image, ImageTk
import threading
import cv2
import os
import numpy as np
import pickle

# Load the trained model from the pickle file
with open('mask_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Class labels
class_labels = ["No Mask", "Mask"]

# Initialize the GUI
window = Tk()
window.title("Face Mask Detection GUI")
window.geometry("480x500")
window.configure(background='white')
window.resizable(0, 0)

# Load GIF image frames for animation
image = Image.open("giphy (1).gif")
frames = []
try:
    while True:
        frames.append(ImageTk.PhotoImage(image))
        image.seek(len(frames))
except EOFError:
    pass

# Create a label with the GIF animation
background_label = Label(window)
background_label.pack()

# Animation control flag
animation_running = True

# Function to animate the GIF frames in a separate thread
def animate():
    index = 0
    while animation_running:
        frame = frames[index]
        background_label.configure(image=frame)
        index = (index + 1) % len(frames)
        background_label.update()
        window.after(100)

# Start the animation in a separate thread
animation_thread = threading.Thread(target=animate, daemon=True)
animation_thread.start()

# Function to upload a file
def file_upload():
    file_path = filedialog.askopenfilename(
        parent=window, initialdir=os.getcwd(), title="Please select an image:",
        filetypes=(("Image files", "*.jpg;*.jpeg;*.png"), ("All files", "*.*"))
    )
    return file_path

# Function for face mask detection
def face_mask_detection():
    try:
        img_path = file_upload()
        if not img_path:
            messagebox.showinfo("Error", "No file selected!")
            return

        # Read and preprocess the image
        img = cv2.imread(img_path)  # Read the image in BGR format
        img_resized = cv2.resize(img, (224, 224))  # Resize to (224, 224)
        img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img_normalized = img_rgb / 255.0  # Normalize pixel values to [0, 1]
        img_reshaped = np.expand_dims(img_normalized, axis=0)  # Add batch dimension

        # Perform prediction
        prediction = model.predict(img_reshaped)
        confidence = float(prediction[0][0])  # Extract scalar value for confidence
        label_index = int(confidence > 0.5)  # Threshold for classification
        label = class_labels[label_index]

        # Display the prediction as a pop-up message
        if label_index == 1:  # Mask
            messagebox.showinfo("Result", f"Face Mask ON :) (Confidence: {confidence:.2f})")
        else:  # No Mask
            messagebox.showinfo("Result", f"No Face Mask :( (Confidence: {confidence:.2f})")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Function to display help message
def help_message():
    messagebox.showinfo("Help", "Upload an image for face mask detection.\n"
                                "Make sure the image contains a clear face.")

# Function to safely quit the application
def quit_application():
    global animation_running
    animation_running = False  # Stop the animation
    window.destroy()

# GUI Layout
Label(window, text='Face Mask Detection', bg="violet", font=('Algerian', 15)).place(x=10, y=30)
Button(window, text="Select Image", bg="violet", command=face_mask_detection, font=('Algerian', 15)).place(x=290, y=380)
Button(window, text="Help", bg="violet", command=help_message, font=('Algerian', 15)).place(x=50, y=380)
Button(window, text="Quit", bg="red", command=quit_application, font=('Algerian', 15)).place(x=180, y=420)

# Start the main event loop
window.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
